[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/040_encoder_decoder/encoder_decoder.ipynb)

# La arquitectura *Encoder-Decoder*

En posts anteriores hemos visto como podemos utilizar `redes neuronales recurrentes` para [generación de texto](https://sensioai.com/blog/037_charRNN) así como [clasificación de texto](https://sensioai.com/blog/038_clasificacion_texto). En ambas aplicaciones hemos entrenado una red neuronal que alimentamos con una secuencia de texto, ya sean letras o palabras en una frase, a la cual le pedimos a la salida una distribución de probabilidad sobre la diferentes categorías (para el caso de la clasificación) o directamente el vocabulario (para la generación de texto). La principal limitación de estos modelos es que no podemos obtener más que una salida, y es por esto que en el caso de la generación de texto concatenamos la salida en cada instante a las entradas para utilizarlo de nuevo como entradas y obtener así una nueva predicción. En este post vamos a ver cómo podemos implementar modelos que no sólo sean capaces de recibir secuencias a la entrada, sino que también puedan dar secuencias de longitud arbitraria a la salida. Este tipo de modelos se conocen como modelos *sequence to sequence* (o simplemente *seq2seq*) y pueden ser utilizados para tareas tales como la generación de texto, traducción entre idiomas, resumir textos, etc. 

![](https://pytorch.org/tutorials/_images/seq2seq.png)

Un modelo *seq2seq* está formado por dos partes: un `encoder` que recibe la secuencia de entrada y un `decoder` encargado de generar la secuencia de salida. 

> 💡 Este post está basado en el siguiente [tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html), en el que podrás encontrar más información.

## El *dataset*

En este post vamos a ver cómo entrenar este tipo de arquitectura para traducir texto del inglés al castellano. Puedes encontrar este dataset para traducción [aquí](https://www.manythings.org/anki/). Una vez descargados los datos vamos a leer el archivo, separando los pares de frases de cada ejemplo. 

In [28]:
import unicodedata
import re

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def read_file(file, reverse=False):
    # Read the file and split into lines
    lines = open(file, encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines]

    return pairs

In [29]:
pairs = read_file('spa.txt')

In [40]:
import random

random.choice(pairs)

['tom is still working here isn t he ?',
 'tom todavia esta trabajando aca no ?']

Como ya hemos visto en los posts anteriores, necesitamos un `tokenizer`. En este caso, la clase `Lang` se encargará de asignar índice a cada palabra calculando también su frecuencia para, más tarde, poder quedarnos sólo con las palabras más frecuentes. Necesitaremos, además, dos nuevos *tokens* especiales: el token `<eos>` y el token `<sos>` para indicar, respectivamente, el inicio y final de una frase. Más adelante veremos cómo utilizarlos. 

In [69]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
    def indexesFromSentence(self, sentence):
        return [self.word2index[word] for word in sentence.split(' ')]
    
    def sentenceFromIndex(self, index):
        return [self.index2word[ix] for ix in index]

Opcionalmente, también podemos a indicar la longitud máxima de las frases a utilizar así como un conjunto de comienzos de frases que queramos filtrar. Esto lo hacemos únicamente para acelerar el proceso de entrenamiento, trabajando con un conjunto pequeño de datos. Si quieres entrenar un buen traductor no hace falta que filtres los datos.

In [70]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p, lang):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[lang].startswith(eng_prefixes)


def filterPairs(pairs, lang=0):
    return [pair for pair in pairs if filterPair(pair, lang)]

In [71]:
def prepareData(file, filter=False, reverse=False):
    
    pairs = read_file(file, reverse)
    print(f"Tenemos {len(pairs)} pares de frases")
    
    
    if filter:
        pairs = filterPairs(pairs, int(reverse))
        print(f"Filtramos a {len(pairs)} pares de frases")
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang('eng')
        output_lang = Lang('spa')
    else:
        input_lang = Lang('spa')
        output_lang = Lang('eng')
    
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
                           
    print("Longitud vocabularios:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
                           
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('spa.txt', True, False)
                           
random.choice(pairs)

Tenemos 124547 pares de frases
Filtramos a 7685 pares de frases
Longitud vocabularios:
spa 2685
eng 3883


['i m clumsy .', 'soy torpe .']

Una vez construidos los dos vocabularios, podemos obtener los índices a partir de una frase, y viceversa, de la siguiente manera.

In [74]:
output_lang.indexesFromSentence('voy a necesitar tu ayuda .')

[92, 50, 2459, 115, 718, 4]

In [75]:
output_lang.sentenceFromIndex([92, 50, 2459, 115, 718, 4])

['voy', 'a', 'necesitar', 'tu', 'ayuda', '.']

Para terminar, las siguientes clases se encargarán de alimentar nuestro modelo *seq2seq* utilizando las clases `Dataset` y `DataLoader` de `Pytorch`.